In [6]:
import os 
import pandas as pd
from bs4 import BeautifulSoup

In [7]:
from io import StringIO

In [8]:
SCORE_DIR = "data/scores"

In [9]:
box_scores = os.listdir(SCORE_DIR)

In [10]:
len(box_scores)

4373

In [11]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [12]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [13]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)),attrs={"id":"line_score"})[0]
    cols = list(line_score.columns)
    cols[0]="team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team","total"]]
    return line_score

In [14]:
def read_stats(soup,team,stat):
    df = pd.read_html(StringIO(str(soup)),attrs={"id": f"box-{team}-game-{stat}"}, index_col = 0)[0]
    df = df.apply(pd.to_numeric,errors="coerce")
    return df

In [15]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season
    

In [16]:
base_cols = None
games = []
for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])
    
    summaries = []
    for team in teams:
        basic = read_stats(soup,team,"basic")
        advanced = read_stats(soup,team,"advanced")
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        summary = pd.concat([totals,maxes])
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        summary = summary[base_cols]
    
        summaries.append(summary)
    
    summary = pd.concat(summaries,axis=1).T
    game = pd.concat([summary,line_score], axis = 1)
    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"
    full_game = pd.concat([game,game_opp], axis = 1)
    full_game["season"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format = "%Y%m%d")
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")
    

100 / 4373
200 / 4373
300 / 4373
400 / 4373
500 / 4373
600 / 4373
700 / 4373
800 / 4373
900 / 4373
1000 / 4373
1100 / 4373
1200 / 4373
1300 / 4373
1400 / 4373
1500 / 4373
1600 / 4373
1700 / 4373
1800 / 4373
1900 / 4373
2000 / 4373
2100 / 4373
2200 / 4373
2300 / 4373
2400 / 4373
2500 / 4373
2600 / 4373
2700 / 4373
2800 / 4373
2900 / 4373
3000 / 4373
3100 / 4373
3200 / 4373
3300 / 4373
3400 / 4373
3500 / 4373
3600 / 4373
3700 / 4373
3800 / 4373
3900 / 4373
4000 / 4373
4100 / 4373
4200 / 4373
4300 / 4373


In [19]:
games_df = pd.concat(games,ignore_index=True)

In [20]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
1,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
2,240.0,240.0,35.0,83.0,0.422,9.0,24.0,0.375,12.0,18.0,...,33.3,30.9,171.0,105.0,MIN,107,1,2017,2017-03-08,False
3,240.0,240.0,42.0,86.0,0.488,5.0,20.0,0.250,18.0,20.0,...,32.0,32.4,158.0,126.0,LAC,91,0,2017,2017-03-08,True
4,265.0,265.0,46.0,100.0,0.460,15.0,39.0,0.385,25.0,32.0,...,20.7,39.9,160.0,125.0,NOP,138,1,2020,2020-01-16,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8741,240.0,240.0,37.0,89.0,0.416,12.0,27.0,0.444,13.0,15.0,...,37.5,30.0,170.0,120.0,SAS,98,0,2017,2017-04-10,True
8742,240.0,240.0,34.0,95.0,0.358,7.0,33.0,0.212,17.0,23.0,...,59.2,34.5,231.0,103.0,LAC,108,1,2019,2018-10-19,False
8743,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
8744,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [21]:
games_df.to_csv("nba_games.csv")